In [1]:
import pickle
import torch
from src.model import Transformer, TransformerConfig
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [4]:
model_folder = "models"

with open(model_folder+"/config.json", "r") as f:
    settings = json.loads(f.read())

with open(model_folder+"/word_data.json", "r") as f:
    word_data = json.loads(f.read())

word_to_index = {k:int(v) for k, v in word_data["word_to_index"].items()}
index_to_word = {int(k):v for k, v in word_data["index_to_word"].items()}

In [42]:
# import torch
# import seaborn as sns

# initial_lr = 0.001
# max_lr = 0.004
# min_lr = 0.0001
# optimizer = torch.optim.Adam([torch.Tensor([[1, 2]]), torch.Tensor([[1]])], lr=initial_lr)

# total_epochs = 200
# warmup_steps = 40
# scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=initial_lr/max_lr, total_iters=warmup_steps)
# scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = total_epochs-warmup_steps, eta_min = min_lr)
# scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[warmup_steps])

# lrs = []
# for i in range(total_epochs):
#     lr = scheduler.optimizer.param_groups[0]['lr']
#     lrs.append(lr)
#     scheduler.step()

# sns.lineplot(lrs)

In [5]:
config = TransformerConfig(vocab_size=settings["vocabulary_size"], max_input_length=settings["max_input_length"], num_heads=settings["num_heads"], num_blocks=settings["num_blocks"], embedding_size=settings["embedding_size"])
transformer = Transformer(config)
transformer.load_state_dict(torch.load(model_folder+"/model.pt", map_location=torch.device(device)))
transformer.to(device)
transformer.train()
print("Model loaded")

Model loaded


In [98]:
def generate_next_token(tokens=None, sample=False, temp=1, top_k=1):
    if sample:
        transformer.eval()
    else: 
        transformer.train()
    if tokens == None:
        tokens = [len(index_to_word)-1]
    x = torch.LongTensor([tokens]).to(device)
    with torch.no_grad():
        y = transformer(x)/temp
        # Retrieve prediction of last token only
        # Don't allow the model to generate <unknown> tokens
        y = y[:, -1, :y.shape[2]-1].view(-1).detach().cpu()
    #print("Probability of sampling a top 100 word:", torch.sum(torch.topk(torch.nn.functional.softmax(y, dim=0), 100).values).item())
    if not sample:
        y = torch.nn.functional.softmax(y, dim=0)
        pred = y.argmax(dim=-1).view(-1)
    else:
        #torch.topk(a.flatten(), 3).indices
        if top_k < 1:
            lowest_indices = torch.topk(y, int(len(y)*(1-top_k)), largest=False).indices
            y[lowest_indices] = float("-inf")
        y = torch.nn.functional.softmax(y, dim=0)
        #print("Probability of sampling a top 100 word:", torch.sum(torch.topk(y, 100).values).item())
        dist = torch.distributions.categorical.Categorical(probs=y)
        pred = dist.sample([1])
    next_word = pred.item()
    return next_word

def print_sentence(words):
    print(" ".join([index_to_word[word] for word in words]))

def generate_sentence(start=None, sample=False, length=50, temp=1, top_k=0.8):
    if start == None:
        sentence = []
    else:
        words = start.split(" ")
        sentence = [word_to_index[x] for x in words]
    
    while len(sentence)<length:
        if len(sentence) < settings["max_input_length"]:
            input_sentence = sentence
        else:
            input_sentence = sentence[-settings["max_input_length"]:]
        next_word = generate_next_token(input_sentence, sample, temp, top_k)
        sentence += [next_word]
    
    print_sentence(sentence)


In [99]:
for i in range(1):
    generate_sentence("what is", length=30, sample=True, temp=0.05, top_k=0.02)

what is man ? bassanio . i weep , sir , his life is a married dog , and god made thee merry report ! bassanio . he needs must


In [100]:
for i in range(10):
    generate_sentence("i", length=30, sample=True, temp=0.05, top_k=0.02)

i long to live no more hard than have i my suit , and these bitter slaves are but till i thought she had any man made and duty to
i warrant you , be satisfied . king henry . madam , i say tomorrow is it .— pardon me , sovereign please your highness defiance . queen margaret .
i say , though best time , yet , ill write it for you . pistol . where is your fellow ? give me too much of man . bullcalf
i am never able to shake the matter fort . lady macbeth . as i am , had thou hadst little fear between his master . enter lady macbeth .
i die straight how to love you . richard . who can be gracious here ? anne . ay , madam . anne . nothing but give me fortune .
i must acquaint you presently . exit . brabantio . look , here comes my kindred . enter roderigo and roderigo . welcome , iago . how now , roderigo
i now , not use love ; see , hear me speak . i am noble , and his bastard daughter , young titus . now , father , for
i wish you wise but same . be dog , and by the good gods , theres thre

In [101]:
for i in range(5):
    generate_sentence("and", False, 50)

and i will be sworn to me . i am sorry i shall be glad i have done . i will not be my father . king . i have a woman . rosaline . i will not be satisfied . berowne . i will not be sworn to wed
and all the rest . enter a messenger . messenger . my lord , the king , my lord of norfolk , i have sent for you , and i will not leave you to give me leave to be a man . king . i will not be patient
and the king , and the rest , and the rest . king henry . i am sorry that we have power to have mercy for thee . king henry . then , farewell . exeunt . scene iii . london . the palace . enter king henry , warwick
and the king . king henry . i am sorry that i have . king john . and i will not be satisfied . king henry . o , i will have thee to be sworn . exeunt . scene ii . a plain near the field . enter king
and the king . king henry . i will , my lord of gloucester , to know the news . king richard . i will not stay tonight . exeunt king henry and derby . king henry . i will not come to me . i will

Examples

- [if thou didst] not , thou art a fool . i am a fool , and thou art a fool . thou art a fool . clown . i am
- [i] have a woman , and i have a good heart . but i will be a man of mine , and i will not be satisfied .
- [i] am a man , and i am sure of it . i have said , and i will not be sworn to thee . i am a king ,
- [you are] a merciful old wench . i am not able to wait upon my friend . i am glad to have merry a launcelet , if i had a
- [you are] a beauteous blossom , and i will encounter thee with my nan . i am a proper friend . anne . i would not play a fool ,
- i am not drunk for certain , nor i am not of any part of my sex . i am sorry for neither
- [you are] in the field . enter a messenger . messenger . news , madam , news ! cleopatra . what news ? messenger . the news ? messenger . the king is gone . messenger . the lord mortimer is coming . antony . the noble antony is come
- [like the sky] , and the wind and the moon . o , what a one that is in the other ?
- [i] am a gentleman of love , and a man of all the world .